In [ ]:
import pandas as pd
from modules import data
from modules import metrics
from modules import valuation
from modules import visualisation
from modules import csv_dowloader

In [ ]:
csv_dowloader.download_csv("googl")

In [ ]:
# define parameters
CURRENT_PRICE = 194
BETA = 0.34

CAP_GROWTH = 1.15
MARKET_RETURN = 0.1
RISK_FREE_RETURN = 0.035

In [ ]:
#get data
df = data.get_data()

In [ ]:
df.columns

# profitability

In [ ]:
visualisation.visualise_profitability(df)

In [ ]:
visualisation.visualise_financial_health(df)

In [ ]:
#get metrics
SHARE_COUNT = metrics.shares(df)
WACC = metrics.wacc(df,
                     CURRENT_PRICE,
                     market_return_rate=MARKET_RETURN,
                     risk_free_rate=RISK_FREE_RETURN, 
                     beta=BETA)


# get past 4 years performance history

In [ ]:
years_to_forecast = 4
OCF_History = list(df.iloc[:years_to_forecast]["Operating_Cash_Flow"])
Net_Income_History = list(df.iloc[:years_to_forecast]["Net_Income_Common"])
FCF_History = list(df.iloc[:years_to_forecast]["Free_Cash_Flow"])
Dividends_History = list(df.iloc[:years_to_forecast]["dividend_per_share"])

In [ ]:
# Net_Income_History[-1] = 0.01

# Valuations

### Operating cash flow

In [ ]:
OCF_YOY_GROWTH = valuation.get_ave_yoy_growth_DCF_value(history=OCF_History,
                                       cap_growth=CAP_GROWTH, 
                                       wacc=WACC, 
                                       no_of_shares=SHARE_COUNT)

OCF_CAGR_GROWTH = valuation.get_cagr_growth_DCF_value(history=OCF_History,
                                    cap_growth=CAP_GROWTH, 
                                    wacc=WACC, 
                                    no_of_shares=SHARE_COUNT)

In [ ]:
print(f"""Value based on YOY OCF projection: {OCF_YOY_GROWTH}
Value based on CAGR OCF projection: {OCF_CAGR_GROWTH}""")

### Net income

In [ ]:
NI_YOY_GROWTH = valuation.get_ave_yoy_growth_DCF_value(history=Net_Income_History,
                                       cap_growth=CAP_GROWTH, 
                                       wacc=WACC, 
                                       no_of_shares=SHARE_COUNT)

NI_CAGR_GROWTH = valuation.get_cagr_growth_DCF_value(history=Net_Income_History,
                                    cap_growth=CAP_GROWTH, 
                                    wacc=WACC, 
                                    no_of_shares=SHARE_COUNT)

In [ ]:
print(f"""Value based on YOY Net Income projection: {NI_YOY_GROWTH}
Value based on CAGR Net Income projection: {NI_CAGR_GROWTH}""")

### FCF 

In [ ]:
FCF_YOY_GROWTH = valuation.get_ave_yoy_growth_DCF_value(history=FCF_History,
                                       cap_growth=CAP_GROWTH, 
                                       wacc=WACC, 
                                       no_of_shares=SHARE_COUNT)

FCF_CAGR_GROWTH = valuation.get_cagr_growth_DCF_value(history=FCF_History,
                                    cap_growth=CAP_GROWTH, 
                                    wacc=WACC, 
                                    no_of_shares=SHARE_COUNT)
FCF_PERPETUAL = valuation.perpetual_FCF_growth_valuation(history = FCF_History, 
                                                        wacc=WACC, 
                                                        no_of_shares=SHARE_COUNT)

In [ ]:
print(f"""Value based on YOY FCF projection: {FCF_YOY_GROWTH}
Value based on CAGR FCF projection: {FCF_CAGR_GROWTH}
Value based on perpetual growth projection: {FCF_PERPETUAL}""")

In [ ]:
### Dividends

In [ ]:
try:
    DIVIDENDS_PERPETUAL = valuation.perpetual_dividend_growth_valuation(history = Dividends_History, 
                                                        wacc=WACC)
except ZeroDivisionError:
    DIVIDENDS_PERPETUAL = 0

In [ ]:
print(f"Value based on perpetual dividends growth projection: {DIVIDENDS_PERPETUAL}")

# troubleshoot

In [ ]:
print(f"debt weightage: {metrics.debt_weight(df, current_share_price=CURRENT_PRICE)}")
print(f"debt cost: {metrics.debt_cost(df)}")
print(f"debt cost: {metrics.debt_cost(df)}")
print("\n")
print(f"equity weightage: {metrics.equity_weight(df, current_share_price=CURRENT_PRICE)}")
print(f"equity cost: {metrics.equity_cost(market_return_rate=MARKET_RETURN,risk_free_rate=RISK_FREE_RETURN,beta=BETA)}")
